In [ ]:
import numpy as np 
import pandas as pd
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
import gc

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,auc


In [ ]:
dtypes = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float16',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int8',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float32',
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float16',
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float16',
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float32',
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float32',
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float16',
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float16',
        'Census_InternalPrimaryDisplayResolutionVertical':      'float16',
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float32',
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }

In [ ]:
train = pd.read_csv("../input/microsoft-malware-prediction/train.csv",dtype=dtypes,
                    usecols = 
                    ["HasDetections","SmartScreen","AVProductStatesIdentifier","CountryIdentifier",
                     "AVProductsInstalled","Census_OSVersion","EngineVersion","AppVersion"])
# sample_submission = pd.read_csv("../input/microsoft-malware-prediction/sample_submission.csv")
# test = pd.read_csv("../input/microsoft-malware-prediction/test.csv")





In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
train.get

In [ ]:
train.describe()

In [ ]:
train.info()

In [ ]:
features = ['EngineVersion','AppVersion','AVProductStatesIdentifier','AVProductsInstalled','CountryIdentifier','SmartScreen','Census_OSVersion']
cate = ['EngineVersion','AppVersion','SmartScreen','Census_OSVersion']
label = ['HasDetections']
train.dropna(inplace=True)
y_train = train["HasDetections"]
train.drop(["HasDetections"],axis = 1,inplace = True)


In [ ]:
X_train,X_test,y_train,y_test = train_test_split(train,y_train)


In [ ]:
lgb_train_data = lgb.Dataset(X_train, label=y_train, feature_name=features, categorical_feature=cate,free_raw_data=False)
lgb_test_data = lgb.Dataset(X_test, label = y_test,feature_name = features, categorical_feature=cate,free_raw_data=False)

In [ ]:
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

In [ ]:
params = {}
params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.5
params['num_leaves'] = 10
params['min_data'] = 50
params['max_depth'] = 10

In [ ]:
num_round = 120
bst = lgb.train(params, lgb_train_data, num_round,valid_sets = [lgb_test_data],early_stopping_rounds=12)

In [ ]:
y_pred = bst.predict(X_test)
print(accuracy_score(y_pred.round(),y_test))

In [ ]:
# del train,y_train
gc.collect() 
test_data = pd.read_csv("../input/microsoft-malware-prediction/test.csv",dtype = dtypes,usecols = ["MachineIdentifier","SmartScreen","AVProductStatesIdentifier","CountryIdentifier","AVProductsInstalled","Census_OSVersion","EngineVersion","AppVersion"])

In [ ]:
test_data.loc[1]

In [ ]:
test_data.head()

In [ ]:
test_data.loc[1][1:]

In [ ]:
test_data.isnull().sum()

In [ ]:
test_data.fillna(test_data.loc[1][1:],inplace = True,axis = 0)


In [ ]:
X_test = test_data[features]
y_pred = bst.predict(X_test)


In [ ]:
sub = pd.DataFrame({"MachineIdentifier":test_data.MachineIdentifier})


In [ ]:
sub["HasDetections"] = y_pred


In [ ]:
sub.to_csv('final.csv',index = False)

In [ ]:
references :- https://www.kaggle.com/karanjakhar/malware-detection